## Wikipedia データをロード

In [ ]:
# from app.auto_topic.infra.wikidb import WikiDb, WikiRecord
# from app.auto_topic.component.models.model import TextSequences
# 
# # n_limit: int = -1
# n_limit: int = 10
# mode: str = "train"
# wdb = WikiDb(mode=mode)
# records = wdb.select(n_limit=n_limit)
# X: TextSequences = [WikiRecord(*rec).paragraph.splitlines() for rec in records]

In [ ]:
# len(X)

## ダウンロードしたニュース記事をロード

In [1]:
import json

with open("data2/bk/2/news.json", "r") as f:
    news = json.load(f)


In [2]:
n = len(news)
n

34845

In [3]:
# import numpy
# 
# idx = numpy.random.randint(0, n)
# html_text = news[idx]["html"]
# html_text[:1000]

'<!DOCTYPE html><html lang="ja" data-service-name="kite-article"><head><meta charSet="utf-8"/><meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1"/><script>LUX=(function(){var a=("undefined"!==typeof(LUX)&&"undefined"!==typeof(LUX.gaMarks)?LUX.gaMarks:[]);var d=("undefined"!==typeof(LUX)&&"undefined"!==typeof(LUX.gaMeasures)?LUX.gaMeasures:[]);var j="LUX_start";var k=window.performance;var l=("undefined"!==typeof(LUX)&&LUX.ns?LUX.ns:(Date.now?Date.now():+(new Date())));if(k&&k.timing&&k.timing.navigationStart){l=k.timing.navigationStart}function f(){if(k&&k.now){return k.now()}var o=Date.now?Date.now():+(new Date());return o-l}function b(n){if(k){if(k.mark){return k.mark(n)}else{if(k.webkitMark){return k.webkitMark(n)}}}a.push({name:n,entryType:"mark",startTime:f(),duration:0});return}function m(p,t,n){if("undefined"===typeof(t)&&h(j)){t=j}if(k){if(k.measure){if(t){if(n){return k.measure(p,t,n)}else{return k.measure(p,t)}}else{return k.measure(p)}}else

In [ ]:
# with open("data/sample.html", "w") as f:
#     f.write(html_text)


In [4]:
from bs4 import BeautifulSoup

def _filter_text(body) -> str:
    soup: BeautifulSoup = BeautifulSoup(body, "lxml")
    for tg in ["script", "noscript", "meta"]:
        try:
            soup.find(tg).replace_with(" ")
        except Exception:
            # NOTE: Not Found `tg` tag
            pass
    return soup.get_text("\n\n")



In [5]:
# text = _filter_text(html_text)
# text[:200]

' \n\n \n\n「多島美」めでる展望台\u3000まさに絵画の世界: 日本経済新聞\n\n日本経済新聞\n\n朝刊・夕刊\n\nLIVE\n\nMyニュース\n\n日経会社情報\n\n人事ウオッチ\n\n日経ビジネス\n\nお申し込み\n\nログイン\n\nメニュー\n\n検索\n\nトップ\n\n速報\n\nアクセスランキング\n\nトピック一覧\n\n人事\n\nおくやみ\n\nプレスリリース\n\nビジュアルデータ\n\n写真\n\n映像\n\n社説\n\n経済指標・統計\n\n連載\n\n話題のトピッ'

In [6]:
import re

def clean_text(text: str):
    contents = []
    for txt in re.split(r"(。|\n)", text):
        txt = txt.strip().replace("\u200b", "").replace("\u3000", " ")
        txt = re.sub(r"\n+", "\n", txt)
        txt = re.sub(r"([\W])\1+", " ", txt)
        if not txt:
            continue
        # contents.append(txt)
        # contents.extend(txt.split("\n"))
        contents.append(txt.split("\n")[-1])
    return contents


In [7]:
# texts = clean_text(text)
# len(texts)
# # texts[:20]


1673

In [8]:
cleaned_texts = []
for itm in news:
    html = itm["html"]
    text = _filter_text(html)
    texts = clean_text(text)
    cleaned_texts.append(texts)

/usr/local/lib/python3.10/dist-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [9]:
len(cleaned_texts)

34845

In [10]:
import joblib

joblib.dump(cleaned_texts, "cleaned_texts.gz", compress=("gzip", 3))


['cleaned_texts.gz']

In [12]:
from app.auto_topic.domain.models.tokenizer import TokenizerWord

tokenizer = TokenizerWord(use_stoppoes=False, filterpos=[], use_orgform=False)

In [13]:
tidy_sentences = []
for idx, texts in enumerate(cleaned_texts):
    for text in texts:
        tokenized = tokenizer([[text]])
        if not tokenized:
            continue
        snt = tokenized[0]
        if len(snt) < 3:
            continue
        # print(idx, snt)       # for debugging
        tidy_sentences.append("".join(snt))


In [ ]:
len(tidy_sentences)

In [14]:
joblib.dump(tidy_sentences, "tidy_sentences.gz", compress=("gzip", 3))


['tidy_sentences.gz']

In [ ]:
# pure_sentences = set(tidy_sentences)
# len(pure_sentences)


In [ ]:
# # n = len(pure_sentences)
# # idx = numpy.random.randint(0, n)
# list(pure_sentences)[idx]

In [ ]:
# from app.auto_topic.component.models.pipeline import Pipeline
# from app.auto_topic.component.models.vectorizer import VectorizerWord2vec
# 
# pipe_vectorizer = Pipeline(
#     steps=[
#         (
#             TokenizerWord(
#                 use_stoppoes=False, filterpos=["名詞", "動詞"], use_orgform=True
#             ),
#             None,
#         ),
#         (VectorizerWord2vec(
#             vector_size=128,
#             sg=1,
#             max_vocab_size=1000 * 1000,
#             min_count=1,
#             window=7,
#             epochs=50,
#         ), None),
#     ]
# )


In [ ]:
# vectorizer:VectorizerWord2vec = pipe_vectorizer.get_model(-1)
# vectorizer.params

In [ ]:
# from app.auto_topic.component.simple_logger import log_info
# 
# X.extend([[s] for s in pure_sentences])
# 
# log_info("Start", "Fit Wiki data")
# pipe_vectorizer.fit(X)
# log_info("End", "Fit Wiki data")


In [ ]:
# vectorizer:VectorizerWord2vec = pipe_vectorizer.get_model(-1)
# len(vectorizer.model.wv.index_to_key)
# 

In [ ]:
# import joblib
# 
# pipe_file = "data/pipe_newswikivec.gz"
# log_info("Start", "Dump Pipeline for Wiki vectorizer")
# joblib.dump(pipe_vectorizer, pipe_file, compress=("gzip", 3))
# log_info("End", "Dump Pipeline for Wiki vectorizer")